In [1]:
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "FinalProject_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [6]:
# Reads the huge train.txt file and creates the .0005 sample.

#df = spark.read.csv('train.txt',sep='\t')
#df_0005 = df.sample(.0005,seed=2615).write.csv('train_0005.txt',sep='\t')

In [65]:
# create dataframe from train_0005.tsv
df_0005 = spark.read.csv('train_0005.txt',sep='\t')

In [73]:
# rename columns from default _c0,_c1... _c26 to ['Label','I0'..[C0]]
# per the readme.
# just run this one once, or reload the data first to run again
old_col_names=df_0005.columns
new_col_names = ['Label']+['I{}'.format(i) for i in range(0,13)]+['C{}'.format(i) for i in range(0,26)]
for old_name,new_name in zip(old_col_names,new_col_names):
    df_0005 = df_0005.withColumnRenamed(old_name,new_name)
# We should probably cast the data now, didn't get this working yet
#   df_0005 = joindf.withcolumn(new_name,joindf(new_name).cast("double"))
df_0005.head(20)

[Row(Label='0', I0='0', I1='1', I2='10', I3='2', I4='4851', I5='139', I6='7', I7='2', I8='53', I9='0', I10='1', I11=None, I12='2', C0='05db9164', C1='403ea497', C2='2cbec47f', C3='3e2bfbda', C4='25c83c98', C5=None, C6='866e2a5e', C7='5b392875', C8='a73ee510', C9='3b08e48b', C10='a15790a3', C11='21a23bfe', C12='a0abbb5b', C13='07d13a8f', C14='e3209fc2', C15='587267a3', C16='d4bb7bd8', C17='a78bd508', C18='21ddcdc9', C19='5840adea', C20='c2a93b37', C21=None, C22='3a171ecb', C23='1793a828', C24='e8b83407', C25='2fede552'),
 Row(Label='0', I0='0', I1='93', I2='21', I3='21', I4='2387', I5='73', I6='7', I7='24', I8='174', I9='0', I10='6', I11=None, I12='21', C0='05db9164', C1='421b43cd', C2='e8c51b7b', C3='29998ed1', C4='25c83c98', C5='fe6b92e5', C6='dc7659bd', C7='0b153874', C8='a73ee510', C9='fed0f64c', C10='e51ddf94', C11='6aaba33c', C12='3516f6e6', C13='b28479f6', C14='e1ac77f7', C15='b041b04a', C16='e5ba7672', C17='2804effd', C18=None, C19=None, C20='723b4dfd', C21=None, C22='3a171ecb',

In [84]:
from pyspark.sql.functions import countDistinct,col
df_counts = df_0005.agg(*(countDistinct(col(c)).alias(c) for c in df_0005.columns)).show()
df_counts.show()

+-----+---+----+---+---+----+----+---+---+----+---+---+---+---+---+---+-----+----+---+---+----+---+---+----+----+-----+----+---+----+-----+---+----+---+---+-----+---+---+----+---+----+
|Label| I0|  I1| I2| I3|  I4|  I5| I6| I7|  I8| I9|I10|I11|I12| C0| C1|   C2|  C3| C4| C5|  C6| C7| C8|  C9| C10|  C11| C12|C13| C14|  C15|C16| C17|C18|C19|  C20|C21|C22| C23|C24| C25|
+-----+---+----+---+---+----+----+---+---+----+---+---+---+---+---+---+-----+----+---+---+----+---+---+----+----+-----+----+---+----+-----+---+----+---+---+-----+---+---+----+---+----+
|    2| 95|1349|358| 70|9769|1030|392| 63|1042|  8| 70| 47|109|261|450|12574|8307| 86|  9|4987|142|  3|5481|2874|11739|2306| 25|3117|10366|  9|1684|770|  3|11156|  9| 14|4732| 44|3621|
+-----+---+----+---+---+----+----+---+---+----+---+---+---+---+---+---+-----+----+---+---+----+---+---+----+----+-----+----+---+----+-----+---+----+---+---+-----+---+---+----+---+----+

